In [13]:
import numpy as np
import numba as nb

import ice_funcs as icef

## Todo:
- ***Figure out if I actually need to hold on to timings... and if yes... how ?***
    - Is the timing array even needed &&&**[do not delete, just comment out]**&&&
- Make relaxSaturationUtilities fully OOP


## Backburner-type-beat:
- make sure everything is numbafied
- Continue commenting in `ice_funcs.py`

## Improvements
- Monopole matching
- correct geometry factors $G_b$ and $H_b$
- Causal timings?

In [14]:
# CONSTANTS

L = 101
W = (L+1)//2
INITIAL_SAT = 1

In [15]:
gu = icef.GeneralUtilities(101) # general utilities

pu = icef.PhysicsUtilities(0.01, 10) # physics utilities

sru = icef.SaturationRelaxationUtilities(101, pu, 200)

In [16]:
default_ice_map = np.full((101, 101), False)
default_ice_map[-1,0] = True
default_ice_map[-2,0] = True
default_ice_map[-3,0] = True
default_ice_map[-3,1] = True

%timeit test_boundary_map
test_boundary_map = gu.construct_boundary_map(default_ice_map)
print(test_boundary_map[-5:,:5])

24.2 ns ± 0.343 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
[[0 0 0 0 0]
 [1 2 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [17]:
sru._distinguish_cells(default_ice_map, test_boundary_map)[1]

array([[97,  0],
       [97,  1]])